In [1]:
# import the package
from mt5linux import MetaTrader5
# connecto to the server
mt5 = MetaTrader5(
    # host = 'localhost' (default)
    # port = 18812       (default)
) 
# use as you learned from: https://www.mql5.com/en/docs/integration/python_metatrader5/
# establish MetaTrader 5 connection to a specified trading account
if not mt5.initialize(login= 69068654, server="MetaQuotes-Demo",password="vpklac8s"):
    print("initialize() failed, error code =",mt5.last_error())
    quit()

print(mt5.terminal_info())
# display data on MetaTrader 5 version
print(mt5.version())

print(mt5.copy_rates_from_pos('VALE3',mt5.TIMEFRAME_M1,0,1000))
# ...
# don't forget to shutdown
#mt5.shutdown()

TerminalInfo(community_account=False, community_connection=False, connected=True, dlls_allowed=False, trade_allowed=False, tradeapi_disabled=False, email_enabled=False, ftp_enabled=False, notifications_enabled=False, mqid=False, build=3683, maxbars=100000, codepage=0, ping_last=185860, community_balance=0.0, retransmission=0.0, company='MetaQuotes Software Corp.', name='MetaTrader 5', language='English', path='C:\\Program Files\\MetaTrader 5', data_path='C:\\Program Files\\MetaTrader 5', ...)
(500, 3683, '16 Apr 2023')
None


In [2]:
import pandas as pd  # install using 'pip install pandas'
from datetime import datetime
import time
def signal(symbol, timeframe, sma_period):
    bars = mt5.copy_rates_from_pos(symbol, timeframe, 1, sma_period)
    bars_df = pd.DataFrame(bars)

    last_close = bars_df.iloc[-1].close
    print(last_close)
    sma = bars_df.close.mean()

    direction = 'flat'
    if last_close > sma:
        direction = 'buy'
    elif last_close < sma:
        direction = 'sell'

    return last_close, sma, direction

In [3]:
SYMBOL = "EURUSD"
TIMEFRAME = mt5.TIMEFRAME_M1
SMA_PERIOD = 10

#mt5.initialize()

while 0:
        # calculating last candle close and simple moving average and checking for trading signal
        last_close, sma, direction = signal(SYMBOL, TIMEFRAME, SMA_PERIOD)


        #print('time: ', datetime.now())
    
        #print('last_close: ', last_close)
        #print('sma: ', sma)
        #print('signal: ', direction)
        #print('-------\n')

        # update every 1 second
        time.sleep(1)

In [2]:
import pandas as pd
import plotly.express as px

#settings 
symbol = "EURUSD"
timeframe = mt5.TIMEFRAME_M1
start_pos = 0
num_bars = 14

In [3]:
bars = mt5.copy_rates_from_pos(symbol,timeframe,start_pos,num_bars)
df = pd.DataFrame(bars)[['time','open','high','low','close']]
df['time'] = pd.to_datetime(df['time'],unit='s')

df

,time,open,high,low,close
0,2023-05-10 20:08:00,1.09705,1.09727,1.09704,1.09720
1,2023-05-10 20:09:00,1.09719,1.09725,1.09709,1.09711
2,2023-05-10 20:10:00,1.09706,1.09711,1.09697,1.09703
3,2023-05-10 20:11:00,1.09703,1.09703,1.09686,1.09688
4,2023-05-10 20:12:00,1.09688,1.09703,1.09683,1.09702
5,2023-05-10 20:13:00,1.09704,1.09708,1.09696,1.09699
6,2023-05-10 20:14:00,1.09699,1.09700,1.09693,1.09693
7,2023-05-10 20:15:00,1.09693,1.09695,1.09682,1.09683
8,2023-05-10 20:16:00,1.09683,1.09688,1.09682,1.09684
9,2023-05-10 20:17:00,1.09683,1.09691,1.09683,1.09683


In [4]:
fig= px.line(df, x='time', y='close', title='EURUSD - Close Prices') #creating a figure using px. Line
display (fig)# showing figure in output

In [5]:
# setting the RSI PerLou
rsi_period = 14
# to calculate RSI, we first need to calculate the exponential weighted aveage gain and Loss during the period
df['gain'] = (df['close'] - df [ 'open']).apply(lambda x: x if x > 0 else 0)
df['loss'] = (df['close'] - df[ 'open']).apply(lambda x: -x if x < 0 else 0)
#here we use the same formula to calculate Exponential Moving Average
df ['ema_gain'] = df['gain'].ewm (span=rsi_period, min_periods=rsi_period).mean()
df ['ema_loss'] = df['loss'].ewm (span=rsi_period, min_periods=rsi_period).mean()
# the Relative Strength is the ratio between the exponential avg gain divided by the exponential avg Loss
df['rs'] = df['ema_gain'] / df['ema_loss']
# the RSI is calculated based on the Relative Strength using the following formula
df['rsi_14'] = 100 - (100 / (df['rs'] + 1))
#displaying the results
display (df[['time', 'rsi_14', 'rs', 'ema_gain', 'ema_loss']])
#plotting the RSI
fig_rsi= px.line(df, x='time', y='rsi_14', title='RSI Indicator')

#RSI commonly uses oversold and overbought Levels, usually at 70 and 30
overbought_level = 70
orversold_level = 30 
# adding oversold and overbought Levels to the plot
fig_rsi.add_hline (y=overbought_level, opacity=0.5)
fig_rsi.add_hline (y=orversold_level, opacity=0.5)
#showing the RSI Figure
display (fig_rsi)

,time,rsi_14,rs,ema_gain,ema_loss
0,2023-05-10 20:08:00,NaN,NaN,NaN,NaN
1,2023-05-10 20:09:00,NaN,NaN,NaN,NaN
2,2023-05-10 20:10:00,NaN,NaN,NaN,NaN
3,2023-05-10 20:11:00,NaN,NaN,NaN,NaN
4,2023-05-10 20:12:00,NaN,NaN,NaN,NaN
5,2023-05-10 20:13:00,NaN,NaN,NaN,NaN
6,2023-05-10 20:14:00,NaN,NaN,NaN,NaN
7,2023-05-10 20:15:00,NaN,NaN,NaN,NaN
8,2023-05-10 20:16:00,NaN,NaN,NaN,NaN
9,2023-05-10 20:17:00,NaN,NaN,NaN,NaN


In [23]:
#Calculating ATR Indicator
atr_period = 14 # defining the atr period to 14
#calculating the range of each candle
df['range'] = df['high'] - df [ 'low']
# calculating the average value of ranges
df ['atr_14'] = df['range'].rolling (atr_period).mean()
display (df[['time', 'atr_14']])
#plotting the ATR Indicator
fig_atr= px.line(df, x= 'time', y='atr_14', title='ATR Indicator')
display (fig_atr)

,time,atr_14
0,2023-05-08 12:13:00,NaN
1,2023-05-08 12:14:00,NaN
2,2023-05-08 12:15:00,NaN
3,2023-05-08 12:16:00,NaN
4,2023-05-08 12:17:00,NaN
...,...,...
195,2023-05-08 15:28:00,0.000194
196,2023-05-08 15:29:00,0.000187
197,2023-05-08 15:30:00,0.000186
198,2023-05-08 15:31:00,0.000186


In [ ]:
import time
def ayush():
    bars = mt5.copy_rates_from_pos(symbol,timeframe,start_pos,num_bars)
    df = pd.DataFrame(bars)[['time','open','high','low','close']]
    df['time'] = pd.to_datetime(df['time'],unit='s')

    # setting the RSI PerLou
    rsi_period = 14
    # to calculate RSI, we first need to calculate the exponential weighted aveage gain and Loss during the period
    df['gain'] = (df['close'] - df [ 'open']).apply(lambda x: x if x > 0 else 0)
    df['loss'] = (df['close'] - df[ 'open']).apply(lambda x: -x if x < 0 else 0)
    #here we use the same formula to calculate Exponential Moving Average
    ema_gain = df['gain'].ewm (span=rsi_period, min_periods=rsi_period).mean()
    ema_loss = df['loss'].ewm (span=rsi_period, min_periods=rsi_period).mean()
    # the Relative Strength is the ratio between the exponential avg gain divided by the exponential avg Loss
    rs = ema_gain / ema_loss
    # the RSI is calculated based on the Relative Strength using the following formula
    rsi_14 = 100 - (100 / (rs + 1))
    #displaying the results
    print(rsi_14)


while 1 :
    ayush()